In [4]:
import glob
import pandas as pd

raw_tsv_file = glob.glob('raw_data/*.tsv')

In [45]:
df_arr = []
for fpath in raw_tsv_file:
    print(fpath)
    df_ = pd.read_csv(fpath, names=range(9))
    # df_.drop(columns=0, inplace=True)
    df_arr.append(df_)

raw_data/Nguyen_outputs.tsv
raw_data/Hau_outputs.tsv
raw_data/Duong_outputs_1500_1999.tsv
raw_data/kiet_output.tsv
raw_data/Khoi_outputs_4122.tsv
raw_data/Duong_outputs_remain.tsv
raw_data/Duong_outputs_0_1499.tsv


In [46]:
df = pd.concat(df_arr)

In [47]:
df[3].unique()

array(['ADDR', 'NUMBER_PREFIX', 'TITLE', 'PRODUCT_NAME_PREFIX',
       'PRODUCT_NAME', 'AMOUNT_PREFIX', 'AMOUNT', 'UNIT_PREFIX', 'UNIT',
       'UPRICE_PREFIX', 'UPRICE', 'SUB_TPRICE_PREFIX', 'SUB_TPRICE',
       'TAMOUNT_PREFIX', 'TAMOUNT', 'TPRICE_PREFIX', 'TPRICE',
       'RECEMONEY_PREFIX', 'RECEMONEY', 'OTHER', 'BILLID_PREFIX',
       'BILLID', 'DATETIME_PREFIX', 'DATETIME', 'CASHIER_PREFIX',
       'CASHIER', 'SHOP_NAME', 'PHONE_PREFIX', 'FPRICE_PREFIX', 'FPRICE',
       'REMAMONEY_PREFIX', 'REMAMONEY', 'PHONE', 'TDISCOUNT_PREFIX',
       'TDISCOUNT', 'ADDR_PREFIX', 'NUMBER', 'FAX_PREFIX', 'FAX',
       'UDISCOUNT_PREFIX', 'UDISCOUNT', 'D', 'fa', 'a', 'ct'],
      dtype=object)

In [48]:
df = df[~df[3].isin(['D', 'fa', 'a', 'ct'])]

In [68]:
df[~df[8].isna()]

,0,1,2,3,4,5,6,7,8
0,mcocr_public_145014mdljs_jpg.rf.b795cdee97fec7...,640,640,ADDR,143,146,430,178,Chợ Sủi Phú Thị Gia Lâm
1,mcocr_public_145014mdljs_jpg.rf.b795cdee97fec7...,640,640,NUMBER_PREFIX,0,182,35,209,Tel:
2,mcocr_public_145014mdljs_jpg.rf.b795cdee97fec7...,640,640,TITLE,138,237,445,281,HÓA ĐƠN BÁN HÀNG
3,mcocr_public_145014mdljs_jpg.rf.b795cdee97fec7...,640,640,PRODUCT_NAME_PREFIX,19,286,127,318,Tên hàng
4,mcocr_public_145014mdljs_jpg.rf.b795cdee97fec7...,640,640,PRODUCT_NAME,14,326,521,360,8936123410032 sữa tắm gội trẻ em lactacyd N0
...,...,...,...,...,...,...,...,...,...
1496,mcocr_public_145014ohtbh_jpg.rf.9ae1b70bebe577...,640,640,OTHER,40,301,339,323,TỔNG TIỀN PHẢI T. TOÁN
1497,mcocr_public_145014ohtbh_jpg.rf.9ae1b70bebe577...,640,640,OTHER,34,322,289,342,TỔNG TIỀN ĐÃ GIẢM
1498,mcocr_public_145014ohtbh_jpg.rf.9ae1b70bebe577...,640,640,OTHER,34,341,237,358,TIỀN KHÁCH TRẢ
1499,mcocr_public_145014ohtbh_jpg.rf.9ae1b70bebe577...,640,640,OTHER,43,360,169,377,TIỀN MẶT


In [75]:
threshold = 10
cnt = 0
removed_cols = []
for x, group in df.groupby(0):
    if len(group[group[8].isna()]) > len(group[8]) // 2:
        removed_cols.append(x)

In [79]:
df = df[~df[0].isin(removed_cols)]

In [108]:
df.reset_index(inplace=True)

In [109]:
df_train = df.sample(frac=0.8)

In [113]:
df_test = df.iloc[df.index.difference(df_train.index)]

In [118]:
df_train.drop(columns='index').to_csv('train.csv', index=False, header=['image','W','H','CLS','tlx', 'tly', 'brx', 'bry', 'text'])
df_test.drop(columns='index').to_csv('val.csv', index=False, header=['image','W','H','CLS','tlx', 'tly', 'brx', 'bry', 'text'])

In [83]:
df.to_csv('final.csv', index=False, header=['image','W','H','CLS','tlx', 'tly', 'brx', 'bry', 'text'])

In [1]:
import pandas as pd

df = pd.read_csv('final.csv')

In [6]:
df= df[~df.text.isna()]

In [9]:
list_file = list(df.groupby('image'))

In [10]:
import random
random.shuffle(list_file)
train=list_file[:int(len(list_file) * 0.8)]
test=list_file[int(len(list_file) * 0.8):]

In [11]:
new_df = {'size': [], 'ner_tags': [], 'tokens': [], 'bboxes': [], 'image': []}

for img_name, sample in train:
    new_df['size'].append(list(zip(sample['W'], sample['H'])))
    new_df['ner_tags'].append(list(sample['CLS']))
    new_df['bboxes'].append(list(zip(sample['tlx'],sample['tly'],sample['brx'],sample['bry'])))
    new_df['image'].append(img_name)
    new_df['tokens'].append(list(sample['text']))

In [14]:
new_df_ = pd.DataFrame(new_df)
new_df_.to_json('train.json', orient='records')

In [15]:
new_df = {'size': [], 'ner_tags': [], 'tokens': [], 'bboxes': [], 'image': []}

for img_name, sample in test:
    new_df['size'].append(list(zip(sample['W'], sample['H'])))
    new_df['ner_tags'].append(list(sample['CLS']))
    new_df['bboxes'].append(list(zip(sample['tlx'],sample['tly'],sample['brx'],sample['bry'])))
    new_df['image'].append(img_name)
    new_df['tokens'].append(list(sample['text']))
new_df_ = pd.DataFrame(new_df)
new_df_.to_json('val.json', orient='records')

In [17]:
df.CLS.unique()

array(['ADDR', 'NUMBER_PREFIX', 'TITLE', 'PRODUCT_NAME_PREFIX',
       'PRODUCT_NAME', 'AMOUNT_PREFIX', 'AMOUNT', 'UNIT_PREFIX', 'UNIT',
       'UPRICE_PREFIX', 'UPRICE', 'SUB_TPRICE_PREFIX', 'SUB_TPRICE',
       'TAMOUNT_PREFIX', 'TAMOUNT', 'TPRICE_PREFIX', 'TPRICE',
       'RECEMONEY_PREFIX', 'RECEMONEY', 'OTHER', 'BILLID_PREFIX',
       'BILLID', 'DATETIME_PREFIX', 'DATETIME', 'CASHIER_PREFIX',
       'CASHIER', 'SHOP_NAME', 'PHONE_PREFIX', 'FPRICE_PREFIX', 'FPRICE',
       'REMAMONEY_PREFIX', 'REMAMONEY', 'PHONE', 'TDISCOUNT_PREFIX',
       'TDISCOUNT', 'ADDR_PREFIX', 'NUMBER', 'FAX_PREFIX', 'FAX',
       'UDISCOUNT_PREFIX', 'UDISCOUNT'], dtype=object)